In [113]:
import pandas as pd
import os
import datetime

excel = pd.ExcelFile('~/Downloads/Presentation.xlsx')
excel.sheet_names
hotel_leads_all_time = excel.parse('hotel leads all time')
net_leads_q1 = excel.parse('Net_Leads Q1-source')

In [123]:
all_leads = pd.DataFrame.from_csv('~/Downloads/Net_Leads.csv', parse_dates=True, index_col=None)

In [132]:
list(all_leads)

['Id',
 'Company Name',
 'Business Vertical',
 'Life Cycle Status',
 'Lead Status',
 'Lead Source',
 'Lead Source Detail',
 'Updated At',
 'Created At',
 'utm_content',
 'Country',
 'Registration Source Info',
 'Original Referrer']

In [131]:
# Check work by looking for nans

all_leads.loc[all_leads['Original Referrer']=='http://m.facebook.com']

In [104]:
trips = [
    ('android-app://com.linkedin.android', 'Advertising', 'LinkedIn - assumed'),
    ('android-app://m.facebook.com', 'Advertising', 'Facebook - assumed'),
    ('http://instagram.com/', 'Advertising', 'Facebook - assumed'),
    ('http://m.facebook.com', 'Advertising', 'Facebook - assumed'),
    ('http://m.facebook.com/', 'Advertising', 'Facebook - assumed'),
    ('https://web.facebook.com/', 'Advertising', 'Facebook - assumed'),
    ('https://www.bing.com/', 'Advertising', 'Facebook - assumed'),
    ('https://www.facebook.com', 'Advertising', 'Facebook - assumed'),
    ('http://www.sojern.com/ebook/creating-a-website-that-converts-sp?utm_source=facebook&utm_medium=cpc&utm_content=ebook-website-converts-sp&utm_campaign=fb-sponsored-posts-gated-global', 'Advertising', 'Facebook - assumed'),
    ('http://www.sojern.com/travel-insights/2017-hotel-report-signup?utm_source=facebook&utm_medium=cpc&utm_content=2017-google-hotel-report&utm_campaign=fb-sponsored-posts-gated-global', 'Advertising', 'Facebook - assumed'),
    ('https://l.facebook.com/', 'Advertising', 'Facebook - assumed'),
    ('https://m.facebook.com/', 'Advertising', 'Facebook - assumed'),
    ('https://www.facebook.com/', 'Advertising', 'Facebook - assumed'),
    ('https://www.google.com/', 'Website', ''),
    ('https://www.linkedin.com/', 'Website', ''),
]

matches = [
    ('http://www.sojern.com/webinars', 'Webinar', ''),
    ('http://www.sojern.com', 'Website', 'Assumed'),
    ('http://solutions.sojern.com', 'Website', 'Assumed'),
    ('http://sojernmail.sojern.com', 'Website', 'Assumed'),
    ('http://es.sojern.com', 'Website', 'Assumed'),
    ('https://www.google.com', 'Advertising', 'Google - Assumed'),
    ('http://hotelmarketing.com', 'Advertising', 'Hotel Marketing.com - Assumed')
]

Reference docs: 

http://stackoverflow.com/questions/37400246/pandas-update-multiple-columns-at-once
http://stackoverflow.com/questions/22291565/pandas-text-matching-like-sqls-like

In [126]:
# Exact matches for more accurate matching

for trip in trips: 
    all_leads.loc[
        (all_leads['Original Referrer']==trip[0]) & 
        (all_leads['Lead Source'].isnull()) , 
        ['Lead Source', 'Lead Source Detail']
    ] = [trip[1], trip[2]]


In [127]:
# Run a "startswith" match so that we can match those random URLs

for match in matches: 
    all_leads.loc[
        (all_leads['Original Referrer'].str.startswith(match[0])) & 
        (all_leads['Lead Source'].isnull()) , 
        ['Lead Source', 'Lead Source Detail']
    ] = [match[1], match[2]]


In [128]:
# Clean up the last stragglers for the lead source data

all_leads.loc[
        all_leads['Lead Source'].isnull() , 
        ['Lead Source', 'Lead Source Detail']
] = ['Other', 'Uncategorized']

In [129]:
# This is to replace the utm_content if it is empty the registration source info form

all_leads.loc[
        (all_leads['utm_content'].isnull()) , 
        'utm_content'
    ] = all_leads['Registration Source Info']

In [130]:
# Export 

all_leads.to_csv('~/Desktop/net_leads_parse.csv')